In [ ]:
import requests
import os
from datetime import datetime, timedelta
from tqdm import tqdm
import time
sys.path.insert(1, '../../src/')
from config import bearer_key
from conversationgrabber import Convo_grabber
from bson import json_util
import collections
import json
import ast

### Grab conversations from twitter API

In [ ]:
BASE_URL = "https://api.twitter.com/2/tweets/search/all?&"
EXTENDED_URL =  "tweet.fields=author_id,conversation_id,created_at,in_reply_to_user_id,referenced_tweets&expansions=author_id,in_reply_to_user_id,referenced_tweets.id&user.fields=name,username"

In [ ]:
a = Convo_grabber(bearer_key = bearer_key, start_date="2021-10-01", end_date="2022-10-01", BASE_URL = BASE_URL, EXTENDED_URL = EXTENDED_URL )
results = a.get_conversation_ids()

### Save results to `json` file

In [ ]:
json_string = json.dumps(results, default=json_util.default)
with open('conversation-ids-climate-change.json', 'w') as outfile:
    json.dump(json_string, outfile)

In [4]:
# open again if needed...
with open('conversation-ids-climate-change.json') as json_file:
    results = ast.literal_eval(json.load(json_file))

### Keep only the conversation id's that indicate an actual conversation
For now, i've selected the `ids` that occur more than 3 times in the data.

In [6]:
ids_ = [item for sublist in [x['conversation_ids'] for x in results ] for item in sublist]# Put all conversation ids in a flat list
conversations = list(set([x for x, y in collections.Counter(ids_).items()]))
print(len(conversations)) # number of conversations


209616


## Retrieve conversations based on ids from the Twitter API
I'm retrieving data in chunks of 100 conversation ids due to time-out issues; additionally--`sleep` time is needed to avoid making to many requests. 

In [ ]:

def get_strptime(date):
        '''takes date (str) and returns dt formats that can be used to query the Twitter API date 
        should be in the format: yyyy-m-d e.g., 2022-9-30'''
        
        return datetime.strptime(f"{date.split('-')[0]}-{date.split('-')[1]}-{date.split('-')[2]}T00:00:00Z", dtformat).strftime(dtformat)

dtformat = '%Y-%m-%dT%H:%M:%SZ'
start_time = get_strptime('2015-1-10')
end_time = get_strptime('2022-10-10')

def get_parameters(start_date, end_date, conversations_id):
    query_params = {'query': f"conversation_id:{conversations_id}", 
    "max_results": "100" , 
    "start_time" : start_time, 
    "end_time" : end_time}
    return query_params


n = 100
chunks = [conversations[i:i + n] for i in range(0, len(conversations), n)]

#f_data = []
number = 65

for chunk in tqdm(chunks[65:]):
    number = number + 1
    data = [] 

    for id in chunk:
        time.sleep(5)
    
        response = requests.get(f"{BASE_URL}{EXTENDED_URL}",  headers={"Authorization": f"Bearer {bearer_key}",  "User-Agent" : "v2FullArchiveSearchPython"}, params = get_parameters(start_time, end_time, id))
        print(f"response status: {response.status_code}")
 
        if response.json() != "{'meta': {'result_count': 0}}":
            data.append(response.json()) 

        json_string = json.dumps(data, default=json_util.default)
        
        with open(f'conversations-new/convo_{number}.json', 'w') as outfile:
            json.dump(json_string, outfile)

    f_data.append(data)

In [ ]:
flat_data = [item for sublist in f_data for item in sublist]
len(flat_data)

### read data in if needed

In [7]:
f_data = []
for i in range(1, 357):

    with open(f'conversations-new/convo_{i}.json') as json_file:
        try: 
            data = json.load(json_file)
            f_data.append(ast.literal_eval(data))
        except:
            print(i)

flat_data = [item for sublist in f_data for item in sublist]
len(flat_data)

8
153


35345

In [8]:
short_list = [ x for x in flat_data if x != flat_data[1]]
print(len(short_list))

6195


### retrieve starting tweets that contain the key word

In [9]:
starting_tweets = []

for x in short_list:
    try:
        for i in x['includes']['tweets']:
            if 'referenced_tweets' not in i:
                if i['text'].lower().count('klimaatverandering') > 1:
                    starting_tweets.append(i)
    except:
        pass

### remove "thread" type starting tweets (in order to only model conversations to a single tweet)

In [10]:
import collections
duplicates = [item for item, count in collections.Counter([x['conversation_id'] for x in starting_tweets]).items() if count > 1]

In [12]:
convos = [x['conversation_id'] for x in starting_tweets]
convos = [x for x in convos if x not in duplicates]
len(convos) == len(set(convos))
len(convos)

39

In [14]:
def get_replies(collection, conId):
    for x in collection:
        if x['data'][0]['conversation_id'] == conId: 
            return x

results = [ get_replies(short_list, conId) for conId in convos ]
results = [x for x in results if x is not None]

In [15]:
def get_author_id(entry_number, results):
    for i in results[entry_number]['includes']['tweets']:
        if 'referenced_tweets' not in i: 
            if i['text'].lower().count('klimaatverandering') > 1:
                return i['author_id'], i['text']

def get_direct_replies(entry_number, results):
    direct_replies = []
    author_id, author_text = get_author_id(entry_number, results)

    for _ in range(0, len(results[entry_number]['data']) ): 
        if results[entry_number]['data'][_]['in_reply_to_user_id'] == author_id :
            direct_replies.append( results[entry_number]['data'][_]) 

    return direct_replies[::-1][:10], author_id, author_text

d_replies = [ get_direct_replies(_, results)[0] for _ in range(0, len(results)) ]
author_id = [ get_direct_replies(_, results)[1] for _ in range(0, len(results)) ]
author_text = [ get_direct_replies(_, results)[2] for _ in range(0, len(results)) ]



In [17]:
def get_camera_ready(entry_number):

    string_text = f"*************************\nAUTHOR FIRST TWEET: {author_id[entry_number]} \n" + author_text[entry_number]
    for _ in range(0, len(d_replies[entry_number])):
        string_text = string_text + "\n\n" + f"REPLY AUTHOR {_}, ID: {d_replies[entry_number][_]['author_id']}, CREATED AT: {d_replies[entry_number][_]['created_at']}\n" 
        string_text = string_text + d_replies[entry_number][_]['text']+ "\n"
    return string_text.encode('utf-8', 'replace').decode()

res_ = [get_camera_ready(i) for i in range(0, len(results)) ]
print(len(res_))

with open('climate-change-conversations.txt', 'w') as f:
    for line in res_:
        f.write(f"r{line}\n")


38
